In [1]:
import open3d as o3d
import numpy as np

from skspatial.objects import Plane, Points
from skspatial.plotting import plot_3d
import matplotlib.pyplot as plt
import matplotlib
import time

import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcds: list[o3d.geometry.PointCloud] = []
plys_dir = 'road_long_towards_plys/'

for fn in os.listdir(plys_dir):
    pcds.append(o3d.io.read_point_cloud(plys_dir + fn))

In [3]:
inliners = []
inliners_colors = []
outliners = []
outliners_err = []

for i, pcd in enumerate(pcds):
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)

    plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=1000)

    plane_model_err, inliers_err = pcd.segment_plane(distance_threshold=0.15,
                                             ransac_n=3,
                                             num_iterations=1000)

    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    outlier_cloud_err = pcd.select_by_index(inliers_err, invert=True)
    outlier_cloud.paint_uniform_color([0.8, 0.5, 0])
    outlier_cloud_err.paint_uniform_color([1, 0, 0])

    inliners.append(np.asarray(inlier_cloud.points))
    inliners_colors.append(np.asarray(inlier_cloud.colors))
    outliners.append(np.asarray(outlier_cloud.points))
    outliners_err.append(np.asarray(outlier_cloud_err.points))

    print(f'Finished {i+1}/{len(pcds)}')

inliners = np.concatenate(inliners)
inliners_colors = np.concatenate(inliners_colors)
outliners = np.concatenate(outliners)
outliners_err = np.concatenate(outliners_err)

In [4]:
inliners_pcd = o3d.geometry.PointCloud()
inliners_pcd.points = o3d.utility.Vector3dVector(inliners)
inliners_pcd.colors = o3d.utility.Vector3dVector(inliners_colors)

outliners_pcd = o3d.geometry.PointCloud()
outliners_pcd.points = o3d.utility.Vector3dVector(outliners)
outliners_pcd.paint_uniform_color([0.8, 0.5, 0.0])

outliners_err_pcd = o3d.geometry.PointCloud()
outliners_err_pcd.points = o3d.utility.Vector3dVector(outliners_err)
outliners_err_pcd.paint_uniform_color([1.0, 0.0, 0.0])

PointCloud with 104413 points.

In [5]:
o3d.visualization.draw_geometries([inliners_pcd, outliners_pcd, outliners_err_pcd])

In [6]:
def flatten(pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
    pts = np.asarray(pcd.points).copy()
    colors = np.asarray(pcd.colors).copy()
    for pt in pts:
        pt[2] = 0.0

    new_pcd = o3d.geometry.PointCloud()
    new_pcd.points = o3d.utility.Vector3dVector(pts)
    new_pcd.colors = o3d.utility.Vector3dVector(colors)

    return new_pcd

flat_inliners = flatten(inliners_pcd)
flat_outliners = flatten(outliners_pcd)
flat_outliners_err = flatten(outliners_err_pcd)

In [7]:
# flat_inliners_filt, _ = flat_inliners.remove_radius_outlier(50, 0.5)
flat_outliners_filt, _ = flat_outliners.remove_radius_outlier(50, 0.5)
flat_outliners_err_filt, _ = flat_outliners_err.remove_radius_outlier(50, 0.5)

In [8]:
o3d.visualization.draw_geometries([flat_inliners, flat_outliners_filt, flat_outliners_err_filt])

In [ ]:
### Taking picture, not working really well

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(flat_inliners)
vis.add_geometry(flat_outliners_filt)
vis.add_geometry(flat_outliners_err_filt)
vis.update_geometry(flat_inliners)
vis.update_geometry(flat_outliners_filt)
vis.update_geometry(flat_outliners_err_filt)
vis.poll_events()
vis.update_renderer()
vis.capture_screen_image('image.png')
vis.destroy_window()

In [10]:
inliners_points = np.asarray(flat_inliners.points).copy()
inliners_colors = np.asarray(flat_inliners.colors).copy()
outliners_points = np.asarray(flat_outliners_filt.points).copy()
outliners_colors = np.asarray(flat_outliners_filt.colors).copy()
outliners_err_points = np.asarray(flat_outliners_err_filt.points).copy()
outliners_err_colors = np.asarray(flat_outliners_err_filt.colors).copy()

In [12]:
inliners_points.min()

-2.429036855697632